In [ ]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore


os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# embeddings 
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# in memory storage (more like database)
vector_store = InMemoryVectorStore(embeddings) # stores our embeddings in memory: for faster searches and retrievals

 ········
Enter API key for OpenAI:  ········
